# Feature Encoding
Feature encoding is the process of converting non-numerical or categorical data into a numerical formate that meachine learning algorithms can understand and process.

we choosee the encoding tewwchnique based on :-
- The type of categorical data (nominal or ordinal),
- The number of unique values in a catagorical columns, and the model we plan to train.

## Types of Categorical Data
1. Nominal Data:
- No Order
  
Examples:
- color(Red, Blue, Green)
- City(Delhi, Mumbai)

2. Ordinal Data:

order exists
Examples:
- size(Small < Medium < Large)
- Rating(Low < Medium < High)

## Why Features Encoding is important:
- Text data can't be used directly in model training.
- proper encoding:
      Improves model accuracy
       Prevents wrong assumptions
      - Makes training stable

# Common Features encoding techniques
- Label encoding
- one-hot encoding
- target encoing
- ordinal encoding
- frequency encoding
- binary encoding
- feature hashmap encoding

In [1]:
import pandas as pd
df=pd.DataFrame({
    "Color":["Red","Green","Blue","Green","Red"],
    "Size":["Small","Medium","Large","Medium","Small"],
    "City":["Hyderbad","Delhi","Mumbai","Delhi","Chennai"],
    "Taregt":[10,20,30,25,12] 
})
df

,Color,Size,City,Taregt
0,Red,Small,Hyderbad,10
1,Green,Medium,Delhi,20
2,Blue,Large,Mumbai,30
3,Green,Medium,Delhi,25
4,Red,Small,Chennai,12


### 1. Label Encoding:
- Each category is assigned a unique integer (eg: "red": 0, "green" : 1)
- Use for ordinal data (inherent order) or with tree based models that are not sensitive to the numerical order

In [2]:
from sklearn.preprocessing import LabelEncoder
df["Color_Label"]=LabelEncoder().fit_transform(df["Color"])
df[["Color","Color_Label"]]

,Color,Color_Label
0,Red,2
1,Green,1
2,Blue,0
3,Green,1
4,Red,2


### 2. One-Hot Encoding
- Create a new binary column for each category with a 1 indicating the presence of that category and 0 otherwise.
- Ideal for nominal data (no inherent order) with low to medium cardinality, and for linear models or neural networks.

In [3]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse_output=False)
encoded=encoder.fit_transform(df[["City"]])
encoded_df=pd.DataFrame(encoded,columns=encoder.get_feature_names_out(["City"]))
encoded_df

,City_Chennai,City_Delhi,City_Hyderbad,City_Mumbai
0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [4]:
one_hot = pd.get_dummies(df["Color"], prefix="Color")
one_hot

,Color_Blue,Color_Green,Color_Red
0,False,False,True
1,False,True,False
2,True,False,False
3,False,True,False
4,False,False,True


In [5]:
ohe = pd.get_dummies(df, columns=["Color"])
ohe

,Size,City,Taregt,Color_Label,Color_Blue,Color_Green,Color_Red
0,Small,Hyderbad,10,2,False,False,True
1,Medium,Delhi,20,1,False,True,False
2,Large,Mumbai,30,0,True,False,False
3,Medium,Delhi,25,1,False,True,False
4,Small,Chennai,12,2,False,False,True


### 3. Ordinal Encoding
- Similar to label encoding, but the integer assignment is based on the specific, predefined order or rank of the categories (e.g., "low" : 0, "medium" : 1, "Hogh" : 2).
- Use specifically when the categorical variables has a natural, meaningful order.

In [6]:
from sklearn.preprocessing import OrdinalEncoder
order = [["Small", "Medium", "Large"]]
df["Size_Ordinal"] = OrdinalEncoder(categories=order).fit_transform(df[["Size"]])
df[["Size","Size_Ordinal"]]

,Size,Size_Ordinal
0,Small,0.0
1,Medium,1.0
2,Large,2.0
3,Medium,1.0
4,Small,0.0


### 4. Target Encoding (Mean)
- Each category is replaced with the mean of the target variable for that category.
- Effective for high-cardinality faetures and when there's a strong relationship between the category and the targett

In [7]:
te = df.groupby("City")["Taregt"].mean()
df["City_Target_Enc"] = df["City"].map(te)
df[["City","City_Target_Enc"]]

,City,City_Target_Enc
0,Hyderbad,10.0
1,Delhi,22.5
2,Mumbai,30.0
3,Delhi,22.5
4,Chennai,12.0


### 5. Frequency Encoding
- Replaces each category with its frequency (count) in the dataset.
- Useful when the frequency of a category is predictive of the target variable.

In [8]:
freq = df["City"].value_counts()
df["City_Freq_Enc"] = df["City"].map(freq)
df[["City", "City_Freq_Enc"]]

,City,City_Freq_Enc
0,Hyderbad,1
1,Delhi,2
2,Mumbai,1
3,Delhi,2
4,Chennai,1


### 6. Binary Encoding
- Categories are first converted to integers, then to binary code, and each bit becomes a separate column.
- Reduces number of columns.
- Better than one-Hot for high cardinality.

In [12]:
import category_encoders as ce
data_binary = ce.BinaryEncoder(cols=["City"]).fit_transform(df["City"])
data_binary

,City_0,City_1,City_2
0,0,0,1
1,0,1,0
2,0,1,1
3,0,1,0
4,1,0,0


### 7. Feature Hashing:
- Uses a hash function to map categories into a fixed number of dimentions, which is memory efficient for extremely high cadinality data.
- Used for high cardinality data where memory comsumption is concern and interpretability is less important.

In [13]:
from sklearn.feature_extraction import FeatureHasher
fh=FeatureHasher(n_features=4, input_type = "string")
hashed = fh.transform([[city] for city in df["City"]])
hashed_df = pd.DataFrame(hashed.toarray())
hashed_df

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,-1.0
2,0.0,0.0,-1.0,0.0
3,0.0,0.0,0.0,-1.0
4,0.0,0.0,1.0,0.0
